<a href="https://colab.research.google.com/github/saiteja-d/spark_course/blob/master/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!ls

sample_data  spark-2.3.1-bin-hadoop2.7	spark-2.3.1-bin-hadoop2.7.tgz


In [8]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [959 kB]
Get:15 http://archive.ub

In [12]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [16]:
!ls


sample_data		   spark-2.3.1-bin-hadoop2.7.tgz
spark-2.3.1-bin-hadoop2.7  spark-2.3.1-bin-hadoop2.7.tgz.1


In [34]:
!wget https://raw.githubusercontent.com/sai-usf/files/master/clustering_dataset.csv

--2019-04-02 22:53:18--  https://raw.githubusercontent.com/sai-usf/files/master/clustering_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694 [text/plain]
Saving to: ‘clustering_dataset.csv.1’

clustering_dataset. 100%[===================>]     694  --.-KB/s    in 0s      

2019-04-02 22:53:18 (73.4 MB/s) - ‘clustering_dataset.csv.1’ saved [694/694]



In [0]:
mv clustering_dataset.csv.1 dataset.csv

In [30]:
!ls

dataset.csv  spark-2.3.1-bin-hadoop2.7	    spark-2.3.1-bin-hadoop2.7.tgz.1
sample_data  spark-2.3.1-bin-hadoop2.7.tgz  spark-warehouse


In [0]:
from pyspark.ml.linalg import Vectors

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
from pyspark.ml.feature import VectorAssembler

#Loading Data

In [0]:
rc = spark.read.csv('dataset.csv',header=True)

In [79]:
rc

DataFrame[col1: string, col2: string, col3: string]

In [88]:
rc.printSchema()

root
 |-- col1: integer (nullable = true)
 |-- col2: integer (nullable = true)
 |-- col3: integer (nullable = true)



In [0]:
from pyspark.sql.types import IntegerType


In [0]:
labels = [('col1', IntegerType()),
 ('col2', IntegerType()),
 ('col3',IntegerType())]

In [83]:
schema = StructType([StructField(x[0],x[1],True) for x in labels])
schema

StructType(List(StructField(col1,IntegerType,true),StructField(col2,IntegerType,true),StructField(col3,IntegerType,true)))

In [0]:
rc = spark.read.csv('dataset.csv',schema=schema)


In [87]:
rc.show(5)

+----+----+----+
|col1|col2|col3|
+----+----+----+
|null|null|null|
|   7|   4|   1|
|   7|   7|   9|
|   7|   9|   6|
|   1|   6|   5|
+----+----+----+
only showing top 5 rows



In [98]:
from pyspark.sql.types import IntegerType
rc = rc.withColumn("col1", rc["col1"].cast(IntegerType()))
rc = rc.withColumn("col2", rc["col2"].cast(IntegerType())
rc = rc.withColumn("col3", rc["col3"].cast(IntegerType()))

SyntaxError: ignored

In [92]:
rc = rc.drop(rc.select['col1 == null'])

TypeError: ignored

In [0]:
sql = """
    Select rc.* FROM rc
    WHERE col1 != null """



In [97]:
sql.show(4)

AttributeError: ignored

#Transformations

In [0]:
vectorAssembler = VectorAssembler(inputCols=["col1","col2","col3"], outputCol="features")

In [78]:
vcluster_df = vectorAssembler.transform(rc)

IllegalArgumentException: ignored

In [57]:
vcluster_df.show()

Py4JJavaError: ignored